In [1]:
 #import modules, packages and libraries
import pandas as pd
import numpy as np
import requests as req
import gzip
import concurrent.futures
import seaborn as sns
sns.set()
from tmdb_calls import doBatch
from multiprocessing import Manager
from io import BytesIO


print('Downloading tables...')

#get film datasets

#set urls
url_title_basics = 'https://datasets.imdbws.com/title.basics.tsv.gz' #film name, year, runtime, genres
url_crew = 'https://datasets.imdbws.com/title.principals.tsv.gz' #actors, actresses, cinematographers, directors (redundant)
url_ratings = 'https://datasets.imdbws.com/title.ratings.tsv.gz' #ratings for films (not all)
url_names = 'https://datasets.imdbws.com/name.basics.tsv.gz' #link table for names against nconst
url_langs = 'https://datasets.imdbws.com/title.akas.tsv.gz' #link table for names against nconst

#download from url
res_title_basics = req.get(url_title_basics).content
res_crew = req.get(url_crew).content
res_ratings = req.get(url_ratings).content
res_names = req.get(url_names).content
res_lang = req.get(url_langs).content

#decompress
title_basics_gzip = gzip.decompress(res_title_basics)
crew_basics_gzip = gzip.decompress(res_crew)
title_ratings_gzip = gzip.decompress(res_ratings)
names_gzip = gzip.decompress(res_names)
title_langs_gzip = gzip.decompress(res_lang)

#read csv into dataframes
titles = pd.read_csv(BytesIO(title_basics_gzip), delimiter='\t',low_memory=False)
crew = pd.read_csv(BytesIO(crew_basics_gzip), delimiter='\t',low_memory=False)
ratings = pd.read_csv(BytesIO(title_ratings_gzip), delimiter='\t',low_memory=False)
names = pd.read_csv(BytesIO(names_gzip), delimiter='\t',low_memory=False)
langs = pd.read_csv(BytesIO(title_langs_gzip), delimiter='\t',low_memory=False)


print('Cleaning data...')

#clean data

# #filter only english films
desired_langs = ['en']
filtered_langs = langs[langs['language'].isin(desired_langs)]
tconsts_filtered_langs = filtered_langs['titleId'].tolist()
desired_regions = ['CA', 'US', 'GB', 'IE', 'AU', 'NZ']
filtered_regions = langs[langs['region'].isin(desired_regions)]
tconsts_filtered_regions = filtered_regions['titleId'].tolist()

#remove unsuitable titles
titles = titles[titles['titleType'] == 'movie']
titles = titles[titles['genres'] != r'\N']
titles['isAdult'] = pd.to_numeric(titles['isAdult'], errors='coerce')
titles = titles[titles['isAdult'] == 0 ]
titles = titles[(titles['startYear'] >= '1955') & (titles['startYear'] != '\\N')]
titles = titles[(titles['tconst'].isin(tconsts_filtered_langs) & (titles['tconst'].isin(tconsts_filtered_regions)))]

#get tconsts for remaining non-film rows, and remove corresponding non-film rows
film_tconsts = titles['tconst'].tolist()
crew = crew[crew['tconst'].isin(film_tconsts)]
ratings = ratings[ratings['tconst'].isin(film_tconsts)]

#set columns to remove from dataset
remove_from_titles = ['originalTitle', 'endYear', 'titleType', 'isAdult']
remove_from_crew = ['ordering','job','characters']
remove_from_ratings = ['numVotes']
remove_from_names = ['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles']

#remove unneeded columns
titles = titles.drop(columns=remove_from_titles)
crew = crew.drop(columns=remove_from_crew)
ratings = ratings.drop(columns=remove_from_ratings)
names = names.drop(columns=remove_from_names)


print('Merging tables...')

#merge relational tables

crew_data = crew.copy()

#merge crew data with names table to get respective names rather than nconst
crew_data['nconst'] = crew_data['nconst'].str.split(', ')
crew_data = crew_data.explode('nconst')
crew_data = pd.merge(crew_data, names, on='nconst', how='left')
crew_data = crew_data.pivot_table(
    index=['tconst'],
    columns=['category'],
    values=['primaryName'],
    aggfunc=lambda x: ', '.join(str(item) for item in x),
).reset_index()

#formaat and restructure columns
crew_data.columns = [' '.join(col).strip() for col in crew_data.columns.values]
crew_data.columns = ['tconst', 'actor', 'actress', 'archive_footage', 'archive_sound', 'cinematographer', 'composer', 'director', 'editor', 'producer', 'production_designer', 'self', 'writer']

#merge datasets for one complete table
crew_data = crew_data.drop(columns=['archive_footage','archive_sound','self', 'production_designer'])
film_data = pd.merge(titles, ratings, on='tconst', how='left')
film_data = pd.merge(film_data, crew_data, on='tconst', how='left')


print('Further cleaning data...')

#remove data-sparse films

columns_check = ['director', 'cinematographer', 'editor', 'writer', 'composer', 'producer']
film_data = film_data[film_data[columns_check].isna().sum(axis=1) < 2] #make smaller to decrease films?

film_data = film_data.dropna(subset=['actor', 'actress'])
film_data = film_data.dropna(subset=['runtimeMinutes'])
film_data = film_data.dropna(subset=['averageRating'])
film_data = film_data.dropna(subset=['genres'])

film_data = film_data[film_data['runtimeMinutes'] != '\\N']
film_data = film_data[film_data['startYear'] != '\\N']
film_data = film_data[film_data['averageRating'] != '\\N']

film_data['cast'] = film_data['actor'] + ', ' + film_data['actress']
film_data.drop(['actor', 'actress'], axis=1, inplace=True)

#add columns for plot and poster path
film_data['plot'] = np.nan
film_data['poster'] = np.nan

print('Films: ' + str(len(film_data)))


print('Fetching plot summaries and posters...')

#get film plot and poster with tmdb api ~ >2hrs


#call api/details for each film with multiprocessing and mutlithreading
if __name__ == '__main__':

    manager = Manager()
    shared_data = manager.Namespace()
    agg_list = []

    batch_size = 1000
    sleep_time = 3

    num_batches = (len(film_data) // batch_size) + 1

    with concurrent.futures.ProcessPoolExecutor(8) as process_executor:

        for i in range(num_batches):

            start_index = i * batch_size
            end_index = (i + 1) * batch_size
            
            shared_data.film_data = film_data.iloc[start_index:end_index]

            future = process_executor.submit(doBatch, shared_data)

            concurrent.futures.wait([future])

            agg_list.append(shared_data.film_data)

            print(f"Batch {i+1}/{num_batches} completed")
                
    film_data = pd.concat(agg_list, ignore_index=True)

#remove films with no plot
film_data = film_data.dropna(subset=['plot'])

final_order = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'cast', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer', 'poster']
film_data = film_data[final_order]

print('Exporting to json...')


#export film data to json

#shuffle order
film_data = film_data.sample(frac=1)
result = film_data.to_json('webpage/films.json' ,orient="records")

print('Total films: ' + str(len(film_data)))

exit(0)

Cleaning data...
Merging tables...
Further cleaning data...
Films: 25527
Fetching plot summaries and posters...
Batch 1/26 completed
Batch 2/26 completed
Batch 3/26 completed
Batch 4/26 completed
Batch 5/26 completed
Batch 6/26 completed
Batch 7/26 completed
Batch 8/26 completed
Batch 9/26 completed
Batch 10/26 completed
Batch 11/26 completed
Batch 12/26 completed
Batch 13/26 completed
Batch 14/26 completed
Batch 15/26 completed
Batch 16/26 completed
Batch 17/26 completed
Batch 18/26 completed
Batch 19/26 completed
Batch 20/26 completed
Batch 21/26 completed
Error in ThreadPoolExecutor: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/movie/tt3960584 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x2a5bf3d10>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Batch 22/26 completed
Batch 23/26 completed
Batch 24/26 completed
Batch 25/26 completed
Batch 26/26 completed
E

: 

In [ ]:
print('Total films: ' + str(len(film_data)))

film_data